In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# 🧠 Neural Forecast

> **NeuralForecast** offers a large collection of neural forecasting models focused on their usability, and robustness. The models range from classic networks like `MLP`, `RNN`s to novel proven contributions like `NBEATS`, `NHITS`, `TFT` and other architectures.

## 🎊 Features

* **Exogenous Variables**: Static, historic and future exogenous support.
* **Forecast Interpretability**: Plot trend, seasonality and exogenous `NBEATS`, `NHITS`, `TFT`, `ESRNN` prediction components.
* **Probabilistic Forecasting**: Simple model adapters for quantile losses and parametric distributions.
* **Train and Evaluation Losses** Scale-dependent, percentage and scale independent errors, and parametric likelihoods.
* **Automatic Model Selection** Parallelized automatic hyperparameter tuning, that efficiently searches best validation configuration.
* **Simple Interface** Unified SKLearn Interface for `StatsForecast` and `MLForecast` compatibility.
* **Model Collection**: Out of the box implementation of `MLP`, `LSTM`, `RNN`, `TCN`, `DilatedRNN`, `NBEATS`, `NHITS`, `ESRNN`, `AutoFormer`, `Informer`, `TFT`, `VanillaTransformer`, and `StemGNN`. See the entire [collection here](https://nixtla.github.io/neuralforecast/models.html).

## Why?

There is a shared belief in Neural forecasting methods' capacity to improve our pipeline's accuracy and efficiency.

Unfortunately, available implementations and published research are yet to realize neural networks' potential. They are hard to use and continuously fail to improve over statistical methods while being computationally prohibitive. For this reason, we created `NeuralForecast`, a library favoring proven accurate and efficient models focusing on their usability.

## 💻 Installation


### PyPI

You can install `NeuralForecast`'s *released version* from the Python package index [pip](https://pypi.org/project/neuralforecast/) with:

```python
pip install neuralforecast
```

(Installing inside a python virtualenvironment or a conda environment is recommended.)


### Conda

Also you can install `NeuralForecast`'s *released version* from [conda](https://anaconda.org/conda-forge/neuralforecast) with:

```python
conda install -c conda-forge neuralforecast
```

(Installing inside a python virtualenvironment or a conda environment is recommended.)

### Dev Mode
If you want to make some modifications to the code and see the effects in real time (without reinstalling), follow the steps below:

```bash
git clone https://github.com/Nixtla/neuralforecast.git
cd neuralforecast
pip install -e .
```

## How to Use

## How to cite